Autor: Karol Szwed\
Data: 17.02.2023 r.\
Projekt: Końcowe zadanie zaliczeniowe - analiza danych z użyciem API dla ZTM w Warszawie

# Cel projektu

TODO: Opisać

# Pobranie danych GPS dla autobusów

TODO: Opisać

In [ ]:
# Because we are working with large objects that are collected over long periods of time, let's define two helper
# functions that will save those objects as binary files, able to be quickly loaded later.

In [5]:
import pickle

"""
 * INPUT:
    - "pickle_filename" - string with the name of the pickle file where the object will be saved;
    - "obj_to_save" - the object that will be saved in the pickle file.
 * FUNCTION: Save an object to a pickle file.
 * OUTPUT: None; function has side effect of creating a pickle file with the saved object.
"""
def save_obj_as_pickle_file(pickle_filename, obj_to_save):
    with open(pickle_filename, 'wb') as f:
        pickle.dump(obj_to_save, f)
        

"""
 * INPUT:
    - "pickle_filename" - string with the name of the pickle file from which the object will be loaded;
    - "obj_to_load" - the object that will be loaded from the pickle file.
 * FUNCTION: Load an object from a pickle file.
 * OUTPUT: The object loaded from the pickle file.
"""
def load_obj_from_pickle_file(pickle_filename):
    with open(pickle_filename, 'rb') as f:
        return pickle.load(f)

In [ ]:
import warsaw_data_api
import datetime
import csv
import time

"""
 * INPUT:
     - "filename" - string with name of the csv file where data will be written to;
     - "_MY_API_KEY" - string with my API key needed for API calls;
     - "timespan" - integer representing amount of seconds for how long the data will be collected;
     - "time_delta_for_buses" - maximum age (in seconds) of the bus GPS data to be considered valid;
     - "update_interval" - time (in seconds) between each data import from the Warsaw Open Data API.
 * FUNCTION: Gather live bus GPS data using API calls and save it to csv file (with header).
 * OUTPUT: None; function has side effect of creating csv file with bus GPS data.
"""
def import_bus_gps_data(filename, _MY_API_KEY, timespan, time_delta_for_buses, update_interval):
    ztm = warsaw_data_api.ztm(apikey=_MY_API_KEY)  # Pass API key
    start_time = time.time()

    print("Starting the data import at", datetime.datetime.now())
    print("Expected to end the data import at", datetime.datetime.now() + datetime.timedelta(seconds=timespan))

    with open(filename, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["lines", "latitude", "longitude", "time", "brigade", "vehicle_number"])  # write header

    while True:
        try:
            buses_all = ztm.get_buses_location()
            with open(filename, 'a', newline='') as file:
                writer = csv.writer(file)
                for bus in buses_all:
                    now = datetime.datetime.now()
                    time_diff = now - bus.time

                    # We want to gather data that is current, so we only collect location data
                    # that is at most 1 min old
                    if time_diff.seconds < time_delta_for_buses:
                        writer.writerow([bus.lines, bus.location.latitude, bus.location.longitude,
                                         bus.time.time(), bus.brigade, bus.vehicle_number])
            time.sleep(update_interval)  # wait for 1 minute between updates
            if time.time() - start_time > timespan:  # If timespan have passed, break the loop
                break
        except Exception:  # If an error occurs, ignore it and try again
            continue
            

filename = "Buses_location_afternoon.csv" # Where GPS data will be saved
_MY_API_KEY = "2620c061-1099-44d9-baab-fdc3a772ab29"  # My API key

timespan = 3600  # 60 minutes
time_delta_for_buses = 60  # 1 minute
update_interval = 60  # 1 minute

import_bus_gps_data(filename, _MY_API_KEY, timespan, time_delta_for_buses, update_interval)

# Pobranie danych o przystankach autobusowych

TODO: Opisać

In [1]:
import json
import pandas as pd
import warsaw_data_api
import csv

file_json = open("bus_stops.json", "r")
json_dict = json.loads(file_json.read())

# Extract the list of dictionaries
dict_list = json_dict['result']

# Convert each dictionary in the list
new_dict_list = []
for d in dict_list:
    new_dict = {item['key']: item['value'] for item in d['values']}
    new_dict_list.append(new_dict)

# Convert the list of new dictionaries into a DataFrame
bus_stops_table = pd.DataFrame(new_dict_list)

# Drop the last column
last_column = bus_stops_table.columns[-1]
bus_stops_table = bus_stops_table.drop(last_column, axis=1)

# Rename the columns
bus_stops_table.columns = ["stop_id", "stop_pole", "stop_name",
                           "street_id", "latitude", "longitude", "direction"]

print(bus_stops_table.head())
print(bus_stops_table.shape)

  stop_id stop_pole stop_name street_id   latitude  longitude       direction
0    1001        01  Kijowska      2201  52.248455  21.044827  al.Zieleniecka
1    1001        02  Kijowska      2201  52.249078  21.044443       Ząbkowska
2    1001        03  Kijowska      2201  52.248928  21.044169  al.Zieleniecka
3    1001        04  Kijowska      2201  52.249969  21.041588       Ząbkowska
4    1001        05  Kijowska      1203  52.250319  21.043861  al.Zieleniecka
(7058, 7)


## Tworzymy słownik zawierający zbiór informację o trasie każdej z linii

Todo: opis

In [3]:
_MY_API_KEY = "2620c061-1099-44d9-baab-fdc3a772ab29"  # my api key
ztm = warsaw_data_api.ztm(apikey=_MY_API_KEY)  # pass api key

bus_line_stops = {}
every_200th = 0 # For printing every 200th entry
for index, row in bus_stops_table.iterrows():
    every_200th += 1
    stop_id = row['stop_id']
    stop_pole = row['stop_pole']
    stop_info = tuple(row.values)
    bus_lines = ztm.get_lines_for_bus_stop_id(stop_id, stop_pole)

    if every_200th == 200:
        print(stop_info)
        print(bus_lines)
        every_200th = 0

    for bus_line in bus_lines:
        if bus_line not in bus_line_stops:
            bus_line_stops[bus_line] = set()
        bus_line_stops[bus_line].add(stop_info)

print(bus_line_stops['504'])

('1055', '01', 'Gorzykowska', '1909', '52.268593', '21.060081', 'Piotra Skargi')
['120', '160', '190', 'N02', 'N11', 'N61']
('1129', '01', 'Bruszewska', '1839', '52.327496', '21.028882', 'Zakłady Zbożowe')
['326', '705', '735', '736']
('1209', '02', 'Powstańców', '1757', '52.277575', '21.115400', 'Czapelska')
['145']
('1296', '01', 'Mochtyńska', '2960', '52.357728', '21.035482', 'Płochocińska')
['326', 'N14']
('1407', '02', 'Wenecka', '0137', '52.356885', '21.133208', 'Struga')
['140', '738', 'L40', 'L45', 'N61']
('1530', '01', 'Januszewicza', '1799', '52.314983', '21.178599', 'Szymańskiego')
['L27']
('1749', '03', 'Piłsudskiego-Kościół', '1757', '52.350916', '21.230183', 'Lipowa')
['L36']
('1864', '01', 'Andersena', '0926', '52.282917', '21.139519', 'Powstańców')
['199']
('2006', '03', 'Międzyborska', '0802', '52.245864', '21.075239', 'Praga-Płd.-Ratusz')
['3', '6', '9', '22', '24', '26']
('2069', '01', 'Pocisk', '1515', '52.252753', '21.151553', 'Marsa-Las')
['183', 'N21']
('2143', '

In [10]:
pickle_filename = 'bus_line_stops.pkl'

# Write the table as a pickle file
save_obj_as_pickle_file(pickle_filename, bus_line_stops)

In [12]:
# Load the table from a pickle file
bus_line_stops = load_obj_from_pickle_file(pickle_filename)
print(bus_line_stops['504'])

{('4037', '02', 'Sasanki', '2802', '52.184076', '20.979223', 'Leżajska'), ('3134', '04', 'Centrum Onkologii', '0703', '52.149980', '21.031813', 'Herbsta'), ('4031', '02', 'Uniwersytet Medyczny', '2802', '52.204780', '20.986330', 'Banacha-Szpital'), ('3013', '02', 'Poleczki', '1801', '52.154080', '21.017960', 'Wyścigi'), ('4037', '03', 'Sasanki', '2019', '52.182977', '20.979998', 'PKP Służewiec'), ('4035', '02', 'Novotel', '2802', '52.190923', '20.981564', 'Baleya'), ('3403', '02', 'Warchałowskiego', '0510', '52.145531', '21.043507', 'Miklaszewskiego'), ('3195', '02', 'os.Wyżyny', '2005', '52.137975', '21.049090', 'Płaskowickiej'), ('3254', '01', 'PKP Służewiec', '2019', '52.181179', '20.986991', 'Postępu'), ('4029', '02', 'Pomnik Lotnika', '1225', '52.216947', '20.990310', 'Filtrowa'), ('3013', '03', 'Poleczki', '0703', '52.152410', '21.019240', 'Herbsta'), ('3301', '01', 'Malinowskiego', '0510', '52.149862', '21.038194', 'Indiry Gandhi'), ('3254', '02', 'PKP Służewiec', '2019', '52.18